## Merging data catalog with our waveform dataframe

In the first notebook, I created a dataframe based on our earthquake waveforms but this dataframe is not enough to make data exploration steps so, I want to get much more information about earthquake waveforms. To do that, I downloaded earthquake catalog based on year 2002.

In [ ]:
!pip install obspy

## Import Libraries and make connection to Google Drive

In [1]:
# Imports
import os
import shutil
import obspy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
# Google Drive connection
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import eartquake catalog and convert it into dataframe

In [3]:
# Read txt file and turn the lines into list
with open('/content/drive/MyDrive/eartquakes_records/20020501_20021231_0_9.0_55_374.txt', 'r', encoding='utf-8', errors='ignore') as f:
  lines = f.readlines()

In [4]:
# Check out first 5 lines
lines[:5]

['No    \tDeprem Kodu\tOlus tarihi\tOlus zamani\tEnlem\tBoylam\tDer(km)\txM\tMD\tML\tMw\tMs\tMb\tTip\tYer\n',
 '000001\t20021231195740\t2002.12.31\t19:57:40.90\t38.0600\t26.6800\t0020\t3.4\t3.4\t0.0\t\t0.0\t0.0\tKe\tEGE DENIZI\n',
 '000002\t20021231035101\t2002.12.31\t03:51:01.00\t36.5400\t28.3500\t0010\t3.2\t3.2\t0.0\t\t0.0\t0.0\tKe\tAKDENIZ\n',
 '000003\t20021231015632\t2002.12.31\t01:56:32.80\t39.2900\t26.0400\t0007\t3.2\t3.2\t0.0\t\t0.0\t0.0\tKe\tMIDILLI ADASI (EGE DENIZI)\n',
 '000004\t20021230225554\t2002.12.30\t22:55:54.80\t37.6100\t27.4300\t0010\t3.0\t3.0\t0.0\t\t0.0\t0.0\tKe\tOZBASI-SOKE (AYDIN) [South 2.2 km]\n']

First line is our header, others are data. Here are the explation of the header;

* No: A unique identifier for each earthquake event.
* Deprem Kodu: Earthquake Code.
* Olus Tarihi: Date of occurrence in the format YYYY.MM.DD (Year.Month.Day).
* Olus Zamani: Time of occurrence in the format HH:mm:ss.ss (Hours:Minutes:Seconds).
* Enlem: Latitude of the earthquake's epicenter.
* Boylam: Longitude of the earthquake's epicenter.
* Der(km): Depth of the earthquake's focus in kilometers.
* xM: Magnitude of the earthquake. The magnitude can be different types, and it's not clear from the provided data which magnitude type is used.
* MD: Duration Magnitude.
* ML: Local Magnitude.
* Mw: Moment Magnitude.
* Ms: Surface Wave Magnitude.
* Mb: Body Wave Magnitude.
* Tip: Type of earthquake.
* Yer: Location or region where the earthquake occurred.

Right now, I have to split each line as a columns and then create catalog dataframe. I don't need `No` column because end of this notebook, I will merge wavefroms and catalog dataframes, so unique identifier is not necessary for this case. Also, `type` column is not necessary because all values are same.

In [5]:
# Create empty lists for each column
earthquake_code = []
date_time = []
latitude = []
longitude = []
depth = []
xM = []
MD = []
ML = []
Mw = []
Ms = []
Mb = []
location = []

In [6]:
# Split each line and appent them into empty list
for i in lines[1:]:
  eq_data = i.split('\t')
  earthquake_code.append(eq_data[1])
  date_time.append(eq_data[2] + ' ' + eq_data[3])
  latitude.append(eq_data[4])
  longitude.append(eq_data[5])
  depth.append(eq_data[6])
  xM.append(eq_data[7])
  MD.append(eq_data[8])
  ML.append(eq_data[9])
  Mw.append(eq_data[10])
  Ms.append(eq_data[11])
  Mb.append(eq_data[12])
  location.append(eq_data[14].strip())

In [7]:
# Add all data into dataframe
catalog_df = pd.DataFrame(
    {'earthquake_code': earthquake_code,
    'date_time': date_time,
    'latitude': latitude,
    'longitude': longitude,
    'depth': depth,
    'xM': xM,
    'MD': MD,
    'ML': ML,
    'Mw': Mw,
    'Ms': Ms,
    'Mb': Mb,
    'location': location}
)

In [8]:
lines[1:5]

['000001\t20021231195740\t2002.12.31\t19:57:40.90\t38.0600\t26.6800\t0020\t3.4\t3.4\t0.0\t\t0.0\t0.0\tKe\tEGE DENIZI\n',
 '000002\t20021231035101\t2002.12.31\t03:51:01.00\t36.5400\t28.3500\t0010\t3.2\t3.2\t0.0\t\t0.0\t0.0\tKe\tAKDENIZ\n',
 '000003\t20021231015632\t2002.12.31\t01:56:32.80\t39.2900\t26.0400\t0007\t3.2\t3.2\t0.0\t\t0.0\t0.0\tKe\tMIDILLI ADASI (EGE DENIZI)\n',
 '000004\t20021230225554\t2002.12.30\t22:55:54.80\t37.6100\t27.4300\t0010\t3.0\t3.0\t0.0\t\t0.0\t0.0\tKe\tOZBASI-SOKE (AYDIN) [South 2.2 km]\n']

In [9]:
catalog_df

,earthquake_code,date_time,latitude,longitude,depth,xM,MD,ML,Mw,Ms,Mb,location
0,20021231195740,2002.12.31 19:57:40.90,38.0600,26.6800,0020,3.4,3.4,0.0,,0.0,0.0,EGE DENIZI
1,20021231035101,2002.12.31 03:51:01.00,36.5400,28.3500,0010,3.2,3.2,0.0,,0.0,0.0,AKDENIZ
2,20021231015632,2002.12.31 01:56:32.80,39.2900,26.0400,0007,3.2,3.2,0.0,,0.0,0.0,MIDILLI ADASI (EGE DENIZI)
3,20021230225554,2002.12.30 22:55:54.80,37.6100,27.4300,0010,3.0,3.0,0.0,,0.0,0.0,OZBASI-SOKE (AYDIN) [South 2.2 km]
4,20021230212328,2002.12.30 21:23:28.40,38.6500,40.6400,0025,3.1,3.1,0.0,,0.0,0.0,BULGURLUK-GENC (BINGOL) [North East 3.5 km]
...,...,...,...,...,...,...,...,...,...,...,...,...
1890,20020501063239,2002.05.01 06:32:39.30,38.6600,31.1500,0010,2.9,2.9,0.0,,0.0,0.0,DEREKARABAG-BOLVADIN (AFYONKARAHISAR) [South W...
1891,20020501045257,2002.05.01 04:52:57.50,38.6800,30.8600,0011,3.0,3.0,0.0,,0.0,0.0,HAMIDIYE-BOLVADIN (AFYONKARAHISAR) [South West...
1892,20020501034041,2002.05.01 03:40:41.40,38.8200,27.5200,0010,2.8,2.8,0.0,,0.0,0.0,BAHADIR-SARUHANLI (MANISA) [West 1.5 km]
1893,20020501025017,2002.05.01 02:50:17.50,38.4700,28.0800,0011,3.1,3.1,0.0,,0.0,0.0,CAFERBEY-SALIHLI (MANISA) [East 0.5 km]


In [10]:
# Convert the dates to datetime objects
catalog_df['date_time'] = pd.to_datetime(catalog_df['date_time'])

In [12]:
# All Mw values is empty so lets delete it
catalog_df.drop('Mw', axis=1, inplace=True)

In [75]:
# Check the dataframe
catalog_df.head()

,earthquake_code,date_time,latitude,longitude,depth,xM,MD,ML,Ms,Mb,location
0,20021231195740,2002-12-31 19:57:40.900,38.0600,26.6800,0020,3.4,3.4,0.0,0.0,0.0,EGE DENIZI
1,20021231035101,2002-12-31 03:51:01.000,36.5400,28.3500,0010,3.2,3.2,0.0,0.0,0.0,AKDENIZ
2,20021231015632,2002-12-31 01:56:32.800,39.2900,26.0400,0007,3.2,3.2,0.0,0.0,0.0,MIDILLI ADASI (EGE DENIZI)
3,20021230225554,2002-12-30 22:55:54.800,37.6100,27.4300,0010,3.0,3.0,0.0,0.0,0.0,OZBASI-SOKE (AYDIN) [South 2.2 km]
4,20021230212328,2002-12-30 21:23:28.400,38.6500,40.6400,0025,3.1,3.1,0.0,0.0,0.0,BULGURLUK-GENC (BINGOL) [North East 3.5 km]


In [76]:
# Get info
catalog_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1895 entries, 0 to 1894
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   earthquake_code  1895 non-null   object        
 1   date_time        1895 non-null   datetime64[ns]
 2   latitude         1895 non-null   object        
 3   longitude        1895 non-null   object        
 4   depth            1895 non-null   object        
 5   xM               1895 non-null   object        
 6   MD               1895 non-null   object        
 7   ML               1895 non-null   object        
 8   Ms               1895 non-null   object        
 9   Mb               1895 non-null   object        
 10  location         1895 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 163.0+ KB


In [77]:
# Save catalog dataframe as a csv file
catalog_df.to_csv('/content/drive/MyDrive/eartquakes_records/20020501_20021231_0_9.0_55_374.csv',
                  index=False)

## Merge catalog and waveforms dataframes

Now, I have earthquake catalog dataframe. Already, I completed waveforms dataframe in the previous notebook and saved it as csv file in my drive. Firstly, I will import waveforms dataframe in this notebook and merge both dataframes by `date` and `time` columns.

In [14]:
# Import waveforms csv file
waveforms_df = pd.read_csv('/content/drive/MyDrive/eartquakes_records/waveworm_data.csv')

In [15]:
# Check the first 5 rows
waveforms_df.head()

,Unnamed: 0,network,station,channel,start-time,end-time,sampling-rate,npts
0,0,KO,YLVX,BHE,2002-05-01T10:45:00.004000Z,2002-05-01T10:49:59.974000Z,100.0,29998
1,1,KO,YLVX,BHE,2002-05-04T23:26:00.004000Z,2002-05-04T23:35:59.974000Z,100.0,59998
2,2,KO,BALB,BHE,2002-05-05T09:21:00.000000Z,2002-05-05T09:31:07.980000Z,50.0,30400
3,3,KO,CTTX,BHE,2002-05-05T09:21:00.004000Z,2002-05-05T09:30:59.964000Z,100.0,59997
4,4,KO,ISKB,BHE,2002-05-05T09:20:54.000000Z,2002-05-05T09:31:01.980000Z,50.0,30400


In [16]:
# Drop the unnamed column
waveforms_df.drop('Unnamed: 0', inplace=True, axis=1)

In [17]:
# Create date and time columns based on start time
waveforms_df['date_time'] = pd.to_datetime(waveforms_df['start-time'].str[:10] + ' ' + waveforms_df['start-time'].str[11:23])

# Check dataframe
waveforms_df.head()

,network,station,channel,start-time,end-time,sampling-rate,npts,date_time
0,KO,YLVX,BHE,2002-05-01T10:45:00.004000Z,2002-05-01T10:49:59.974000Z,100.0,29998,2002-05-01 10:45:00.004
1,KO,YLVX,BHE,2002-05-04T23:26:00.004000Z,2002-05-04T23:35:59.974000Z,100.0,59998,2002-05-04 23:26:00.004
2,KO,BALB,BHE,2002-05-05T09:21:00.000000Z,2002-05-05T09:31:07.980000Z,50.0,30400,2002-05-05 09:21:00.000
3,KO,CTTX,BHE,2002-05-05T09:21:00.004000Z,2002-05-05T09:30:59.964000Z,100.0,59997,2002-05-05 09:21:00.004
4,KO,ISKB,BHE,2002-05-05T09:20:54.000000Z,2002-05-05T09:31:01.980000Z,50.0,30400,2002-05-05 09:20:54.000


In [18]:
# Get info
waveforms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487 entries, 0 to 1486
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   network        1487 non-null   object        
 1   station        1487 non-null   object        
 2   channel        1487 non-null   object        
 3   start-time     1487 non-null   object        
 4   end-time       1487 non-null   object        
 5   sampling-rate  1487 non-null   float64       
 6   npts           1487 non-null   int64         
 7   date_time      1487 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 93.1+ KB


In [19]:
# Create new columns before the merge them
waveforms_df[['earthquake_code', 'latitude', 'longitude', 'depth', 'xM', 'MD', 'ML', 'Ms', 'Mb', 'location']] = None

In [20]:
count = 0
# Loops for merging data
for index, row in waveforms_df.iterrows():
  # Create time range (+2 and -2)
  two_min_before = row['date_time']+timedelta(minutes=-2)
  two_min_after = row['date_time']+timedelta(minutes=+2)
  # Filter catalog dataframe based on earthqauke date time
  match_df = catalog_df[(catalog_df['date_time'] < two_min_after) & (catalog_df['date_time'] > two_min_before)]
  # Add new values into waveforms dataframe
  if len(match_df) == 1:
    waveforms_df.loc[index, 'earthquake_code'] = match_df['earthquake_code'].values[0]
    waveforms_df.loc[index, 'latitude'] = match_df['latitude'].values[0]
    waveforms_df.loc[index, 'longitude'] = match_df['longitude'].values[0]
    waveforms_df.loc[index, 'latitude'] = match_df['latitude'].values[0]
    waveforms_df.loc[index, 'depth'] = match_df['depth'].values[0]
    waveforms_df.loc[index, 'xM'] = match_df['xM'].values[0]
    waveforms_df.loc[index, 'MD'] = match_df['MD'].values[0]
    waveforms_df.loc[index, 'ML'] = match_df['ML'].values[0]
    waveforms_df.loc[index, 'Ms'] = match_df['Ms'].values[0]
    waveforms_df.loc[index, 'Mb'] = match_df['Mb'].values[0]
    waveforms_df.loc[index, 'location'] = match_df['location'].values[0]

In [21]:
waveforms_df.head()

,network,station,channel,start-time,end-time,sampling-rate,npts,date_time,earthquake_code,latitude,longitude,depth,xM,MD,ML,Ms,Mb,location
0,KO,YLVX,BHE,2002-05-01T10:45:00.004000Z,2002-05-01T10:49:59.974000Z,100.0,29998,2002-05-01 10:45:00.004,20020501104538,38.6200,30.9900,0007,3.4,3.4,0.0,0.0,0.0,CUMHURIYET-CAY (AFYONKARAHISAR) [North East 4...
1,KO,YLVX,BHE,2002-05-04T23:26:00.004000Z,2002-05-04T23:35:59.974000Z,100.0,59998,2002-05-04 23:26:00.004,20020504232625,40.7800,30.8600,0009,3.1,3.1,0.0,0.0,0.0,YESILYAYLA-GUMUSOVA (DUZCE) [South West 2.1 km]
2,KO,BALB,BHE,2002-05-05T09:21:00.000000Z,2002-05-05T09:31:07.980000Z,50.0,30400,2002-05-05 09:21:00.000,20020505092209,40.5300,28.3200,0007,4.1,3.9,4.1,0.0,0.0,MARMARA DENIZI
3,KO,CTTX,BHE,2002-05-05T09:21:00.004000Z,2002-05-05T09:30:59.964000Z,100.0,59997,2002-05-05 09:21:00.004,20020505092209,40.5300,28.3200,0007,4.1,3.9,4.1,0.0,0.0,MARMARA DENIZI
4,KO,ISKB,BHE,2002-05-05T09:20:54.000000Z,2002-05-05T09:31:01.980000Z,50.0,30400,2002-05-05 09:20:54.000,20020505092209,40.5300,28.3200,0007,4.1,3.9,4.1,0.0,0.0,MARMARA DENIZI


Finally, I have merged two dataframe but still there is some missing values in the data. I checked all catalog and didn't see the this records. So, I will move this records in another folder and will not use in my project. Let's create our final dataframe for data exploration steps and delete missing records from our main directory.

In [22]:
df = waveforms_df[~waveforms_df.isna().any(axis=1)]
df

,network,station,channel,start-time,end-time,sampling-rate,npts,date_time,earthquake_code,latitude,longitude,depth,xM,MD,ML,Ms,Mb,location
0,KO,YLVX,BHE,2002-05-01T10:45:00.004000Z,2002-05-01T10:49:59.974000Z,100.0,29998,2002-05-01 10:45:00.004,20020501104538,38.6200,30.9900,0007,3.4,3.4,0.0,0.0,0.0,CUMHURIYET-CAY (AFYONKARAHISAR) [North East 4...
1,KO,YLVX,BHE,2002-05-04T23:26:00.004000Z,2002-05-04T23:35:59.974000Z,100.0,59998,2002-05-04 23:26:00.004,20020504232625,40.7800,30.8600,0009,3.1,3.1,0.0,0.0,0.0,YESILYAYLA-GUMUSOVA (DUZCE) [South West 2.1 km]
2,KO,BALB,BHE,2002-05-05T09:21:00.000000Z,2002-05-05T09:31:07.980000Z,50.0,30400,2002-05-05 09:21:00.000,20020505092209,40.5300,28.3200,0007,4.1,3.9,4.1,0.0,0.0,MARMARA DENIZI
3,KO,CTTX,BHE,2002-05-05T09:21:00.004000Z,2002-05-05T09:30:59.964000Z,100.0,59997,2002-05-05 09:21:00.004,20020505092209,40.5300,28.3200,0007,4.1,3.9,4.1,0.0,0.0,MARMARA DENIZI
4,KO,ISKB,BHE,2002-05-05T09:20:54.000000Z,2002-05-05T09:31:01.980000Z,50.0,30400,2002-05-05 09:20:54.000,20020505092209,40.5300,28.3200,0007,4.1,3.9,4.1,0.0,0.0,MARMARA DENIZI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,KO,YLVX,BHE,2002-12-27T14:42:59.954000Z,2002-12-27T14:46:30.944000Z,100.0,21100,2002-12-27 14:42:59.954,20021227144356,40.2200,28.8100,0019,2.6,2.6,0.0,0.0,0.0,BASKOY-NILUFER (BURSA) [North East 4.8 km]
1480,KO,ISKB,BHE,2002-12-29T22:35:58.000000Z,2002-12-29T22:41:07.980000Z,50.0,15500,2002-12-29 22:35:58.000,20021229223623,40.6200,29.0800,0004,2.8,2.8,0.0,0.0,0.0,TESVIKIYE-CINARCIK (YALOVA) [North 0.5 km]
1481,KO,MRMX,BHE,2002-12-29T22:35:59.834000Z,2002-12-29T22:41:00.824000Z,100.0,30100,2002-12-29 22:35:59.834,20021229223623,40.6200,29.0800,0004,2.8,2.8,0.0,0.0,0.0,TESVIKIYE-CINARCIK (YALOVA) [North 0.5 km]
1482,KO,YLVX,BHE,2002-12-29T22:35:59.314000Z,2002-12-29T22:41:00.304000Z,100.0,30100,2002-12-29 22:35:59.314,20021229223623,40.6200,29.0800,0004,2.8,2.8,0.0,0.0,0.0,TESVIKIYE-CINARCIK (YALOVA) [North 0.5 km]


In [209]:
df.to_csv('/content/drive/MyDrive/eartquakes_records/final_earthquake_dataset.csv', index=False)

## Moving unmerged records from main directory

In [67]:
# Check unmerged dataframe
unmerged_df = waveforms_df[waveforms_df.isna().any(axis=1)]
unmerged_df

,network,station,channel,start-time,end-time,sampling-rate,npts,date_time,earthquake_code,latitude,longitude,depth,xM,MD,ML,Ms,Mb,location
15,KO,BALB,BHE,2002-05-08T19:56:00.000000Z,2002-05-08T20:02:09.980000Z,50.0,18500,2002-05-08 19:56:00.000,None,None,None,None,None,None,None,None,None,None
16,KO,CTTX,BHE,2002-05-08T19:56:00.004000Z,2002-05-08T20:01:59.994000Z,100.0,36000,2002-05-08 19:56:00.004,None,None,None,None,None,None,None,None,None,None
17,KO,ISKB,BHE,2002-05-08T19:56:00.000000Z,2002-05-08T20:02:09.980000Z,50.0,18500,2002-05-08 19:56:00.000,None,None,None,None,None,None,None,None,None,None
18,KO,YLVX,BHE,2002-05-08T19:56:00.004000Z,2002-05-08T20:01:59.974000Z,100.0,35998,2002-05-08 19:56:00.004,None,None,None,None,None,None,None,None,None,None
19,KO,BALB,BHE,2002-05-09T01:49:00.000000Z,2002-05-09T01:59:07.980000Z,50.0,30400,2002-05-09 01:49:00.000,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,KO,MRMX,BHE,2002-12-28T09:47:59.254000Z,2002-12-28T09:51:30.244000Z,100.0,21100,2002-12-28 09:47:59.254,None,None,None,None,None,None,None,None,None,None
1479,KO,YLVX,BHE,2002-12-28T09:47:59.394000Z,2002-12-28T09:51:30.384000Z,100.0,21100,2002-12-28 09:47:59.394,None,None,None,None,None,None,None,None,None,None
1483,KO,ISKB,BHE,2002-12-30T10:23:58.000000Z,2002-12-30T10:24:37.980000Z,50.0,2000,2002-12-30 10:23:58.000,None,None,None,None,None,None,None,None,None,None
1484,KO,ISKB,BHE,2002-12-30T10:58:58.000000Z,2002-12-30T11:02:17.980000Z,50.0,10000,2002-12-30 10:58:58.000,None,None,None,None,None,None,None,None,None,None


In [68]:
# Get the start time of unmerged records
start_time_unmerged = unmerged_df['start-time'].tolist()
start_time_unmerged[:5]

['2002-05-08T19:56:00.000000Z',
 '2002-05-08T19:56:00.004000Z',
 '2002-05-08T19:56:00.000000Z',
 '2002-05-08T19:56:00.004000Z',
 '2002-05-09T01:49:00.000000Z']

In [ ]:
# Create an empty list for folder names
unmerged_folder_names = []

# Add the unmerged folder's name into empty list
for i in sorted(os.listdir('/content/drive/MyDrive/eartquakes_records/2002_ordered')):
  record = sorted(os.listdir(f'/content/drive/MyDrive/eartquakes_records/2002_ordered/{i}'))[0]
  st = obspy.read(f'/content/drive/MyDrive/eartquakes_records/2002_ordered/{i}/{record}')
  if str(st[0].stats.starttime) in start_time_unmerged:
    unmerged_folder_names.append(f'{i}')

In [69]:
# Check the shapes
len(unmerged_df), len(unmerged_folder_names)

(453, 453)

In [71]:
# Move the unmerged records into another directory
for i in unmerged_folder_names:
  # Define record and destination path
  record_path = f'/content/drive/MyDrive/eartquakes_records/2002_ordered/{i}'
  destination_path = f"/content/drive/MyDrive/eartquakes_records/unmerged_records_with_catalog/{i}"
  # Copy the directories
  shutil.move(record_path, destination_path)

Let's check out the total records in unmerged folder. It should be 453.

In [74]:
# Check total records in unmerged folders
len(os.listdir('/content/drive/MyDrive/eartquakes_records/unmerged_records_with_catalog/'))

453